In [1]:
import sys
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal

In [2]:
class Particle:
    def __init__(self, init_pose):
        self.pose = init_pose
        
    def motion_update(self, nu, omega, time, noise_rate_pdf):
        ns = noise_rate_pdf.rvs()
        pnu = nu + ns[0]*math.sqrt(abs(nu)/time) + ns[1]*math.sqrt(abs(omega)/time)
        pomega = omega + ns[2]*math.sqrt(abs(nu)/time) + ns[3]*math.sqrt(abs(omega)/time)
        self.pose = IdealRobot.state_transition(pnu, pomega, time, self.pose)

In [3]:
class Mcl:
    def __init__(self, init_pose, num, motion_noise_stds):
        self.particles = [Particle(init_pose) for i in range(num)]
        
        v = motion_noise_stds
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        
    def motion_update(self, nu, omega, time):
        for p in self.particles: p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
        
    def draw(self, ax, elems):
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2]) for p in self.particles]
        vys = [math.sin(p.pose[2]) for p in self.particles]
        elems.append(ax.quiver(xs, ys, vxs, vys, color='blue', alpha=0.5))

In [4]:
class EstimationAgent(Agent):
    def __init__(self, time_interval, nu, omega, estimator):
        super().__init__(nu, omega)
        self.estimator = estimator
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
    def decision(self, observation=None):
        self.estimator.motion_update(self.prev_nu, self.prev_omega, self.time_interval)
        self.prev_nu, self.prev_omega = self.nu, self.omega
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.estimator.draw(ax, elems)

In [5]:
initial_pose = np.array([0, 0, 0]).T 
estimator = Mcl(initial_pose, 100, motion_noise_stds={"nn":0.01, "no":0.02, "on":0.03, "oo":0.04})
a = EstimationAgent(0.1, 0.2, 10.0/180*math.pi, estimator)
estimator.motion_update(0.2, 10.0/180*math.pi, 0.1)
for p in estimator.particles:
    print(p.pose)

[0.01861132 0.00026497 0.02847183]
[0.02446909 0.00015962 0.01304649]
[0.01535682 0.0001026  0.01336149]
[0.02180838 0.00023348 0.02141149]
[1.39276748e-02 6.54017299e-05 9.39155318e-03]
[0.01644658 0.00015301 0.01860583]
[0.02141668 0.00023828 0.02225085]
[0.02304837 0.00022735 0.01972737]
[0.01942919 0.00016815 0.01730842]
[0.02333941 0.00022308 0.01911575]
[0.01908874 0.00025019 0.02621232]
[0.01690758 0.00013327 0.01576454]
[0.01995153 0.00024433 0.02449159]
[0.02397996 0.00025197 0.02101431]
[0.02350277 0.00029328 0.02495551]
[0.01984664 0.00013278 0.01338046]
[0.01893942 0.00025447 0.02687006]
[0.01800121 0.00023228 0.02580628]
[0.0181643  0.00016115 0.01774283]
[0.02032352 0.00015215 0.01497212]
[0.02265129 0.00020404 0.01801485]
[0.01502418 0.00015472 0.0205958 ]
[0.01999174 0.0002042  0.02042814]
[1.65460977e-02 7.55598499e-05 9.13318972e-03]
[0.02247647 0.00012856 0.01143974]
[0.02489204 0.00028707 0.02306417]
[0.01823291 0.00025759 0.02825312]
[0.01987907 0.00033865 0.034067

In [6]:
def trial(motion_noise_stds): 
    time_interval = 0.1 
    world = World(30, time_interval) 

    initial_pose = np.array([0, 0, 0]).T
    estimator = Mcl(initial_pose, 100, motion_noise_stds)
    circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, estimator)
    r = Robot(initial_pose, sensor=None, agent=circling, color="red" )
    world.append(r)

    world.draw()
    
trial({"nn":0.01, "no":0.02, "on":0.03, "oo":0.04})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …